In [8]:
import sys
import os
print(sys.executable)

/Users/Ram/projects/r3 /agent_env/bin/python


In [12]:
import sys
import os
sys.path.insert(0, os.path.abspath(".."))  # goes up from notebook/ to project root

In [19]:
from src.utils import model_loader

ImportError: cannot import name 'GLOBAL_LOGGER' from 'logger' (/Users/Ram/projects/r3 /src/logger/__init__.py)